Este cuaderno es una versión modificada por Diego Pastor Bonet del trabajo original realizado por Bonafilia et al.

Para ver el trabajo original remítanse a: https://github.com/cloudtostreet/Sen1Floods11/blob/master/Train.ipynb

Las secciones modificadas están debidamente señalizadas.

#Código original

The following is an example of how to utilize our Sen1Floods11 dataset for training a FCNN. In this example, we train and validate on hand-labeled chips of flood events. However, our dataset includes several other options that are detailed in the README. To replace the dataset, as outlined further below, simply replace the train, test, and validation split csv's, and download the corresponding dataset.

Authenticate Google Cloud Platform. Note that to run this code, you must connect your notebook runtime to a GPU.

In [ ]:
from google.colab import auth
auth.authenticate_user()

!curl https://sdk.cloud.google.com | bash

!gcloud init

Se han truncado las últimas 5000 líneas del flujo de salida.
google-cloud-sdk/lib/surface/service_directory/services/__init__.py
google-cloud-sdk/lib/surface/service_directory/services/add_iam_policy_binding.py
google-cloud-sdk/lib/surface/service_directory/services/create.py
google-cloud-sdk/lib/surface/service_directory/services/delete.py
google-cloud-sdk/lib/surface/service_directory/services/describe.py
google-cloud-sdk/lib/surface/service_directory/services/get_iam_policy.py
google-cloud-sdk/lib/surface/service_directory/services/list.py
google-cloud-sdk/lib/surface/service_directory/services/remove_iam_policy_binding.py
google-cloud-sdk/lib/surface/service_directory/services/resolve.py
google-cloud-sdk/lib/surface/service_directory/services/set_iam_policy.py
google-cloud-sdk/lib/surface/service_directory/services/update.py
google-cloud-sdk/lib/surface/service_extensions/__init__.py
google-cloud-sdk/lib/surface/service_extensions/lb_service_steering_extensions/__init__.py
google-c

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  16701      0 --:--:-- --:--:-- --:--:-- 16723
OK
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 5,194 kB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../gcsfuse_1.0.1_amd64.deb ...
Unpacking gcsfuse (1.0.1) ...
Setting up gcsfuse (1.0.1) ...


Install RasterIO

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.5 MB/s eta 0:00:00


Define a model checkpoint folder, for storing network checkpoints during training

In [ ]:
%cd /home
!sudo mkdir checkpoints

/home


Download train, test, and validation splits for both flood water. To download different train, test, and validation splits, simply replace these paths with the path to a csv containing the desired splits.

In [ ]:
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_train_data.csv .
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_test_data.csv .
!gsutil cp gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_valid_data.csv .

Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_train_data.csv...
/ [1 files][ 13.3 KiB/ 13.3 KiB]                                                
Operation completed over 1 objects/13.3 KiB.                                     
Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_test_data.csv...
/ [1 files][  4.7 KiB/  4.7 KiB]                                                
Operation completed over 1 objects/4.7 KiB.                                      
Copying gs://sen1floods11/v1.1/splits/flood_handlabeled/flood_valid_data.csv...
/ [1 files][  4.7 KiB/  4.7 KiB]                                                
Operation completed over 1 objects/4.7 KiB.                                      


Download raw train, test, and validation data. In this example, we are downloading train, test, and validation data of flood images which are hand labeled. However, you can simply replace these paths with whichever dataset you would like to use - further documentation of the Sen1Floods11 dataset and organization is available in the README.

In [ ]:
!sudo mkdir files
!sudo mkdir files/S1
!sudo mkdir files/Labels

!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand files/S1
!gsutil -m rsync -r gs://sen1floods11/v1.1/data/flood_events/HandLabeled/LabelHand files/Labels


both the source and destination. Your crcmod installation isn't using the
module's C extension, so checksumming will run very slowly. If this is your
first rsync since updating gsutil, this rsync can take significantly longer than
usual. For help installing the extension, please see "gsutil help crcmod".

Building synchronization state...
Starting synchronization...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_233925_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_103757_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_360519_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_195474_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_23014_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/HandLabeled/S1Hand/Bolivia_379434_S1Hand.tif...
Copying gs://sen1floods11/v1.1/data/flood_events/

Define model training hyperparameters

In [ ]:
LR = 5e-4
EPOCHS = 100
EPOCHS_PER_UPDATE = 1
RUNNAME = "Sen1Floods11"

Define functions to process and augment training and testing images

In [ ]:
import torch
from torchvision import transforms
import torchvision.transforms.functional as F
import random
from PIL import Image

class InMemoryDataset(torch.utils.data.Dataset):

  def __init__(self, data_list, preprocess_func):
    self.data_list = data_list
    self.preprocess_func = preprocess_func

  def __getitem__(self, i):
    return self.preprocess_func(self.data_list[i])

  def __len__(self):
    return len(self.data_list)


def processAndAugment(data):
  (x,y) = data
  im,label = x.copy(), y.copy()

  # convert to PIL for easier transforms
  im1 = Image.fromarray(im[0])
  im2 = Image.fromarray(im[1])
  label = Image.fromarray(label.squeeze())

  # Get params for random transforms
  i, j, h, w = transforms.RandomCrop.get_params(im1, (256, 256))

  im1 = F.crop(im1, i, j, h, w)
  im2 = F.crop(im2, i, j, h, w)
  label = F.crop(label, i, j, h, w)
  if random.random() > 0.5:
    im1 = F.hflip(im1)
    im2 = F.hflip(im2)
    label = F.hflip(label)
  if random.random() > 0.5:
    im1 = F.vflip(im1)
    im2 = F.vflip(im2)
    label = F.vflip(label)

  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  im = torch.stack([transforms.ToTensor()(im1).squeeze(), transforms.ToTensor()(im2).squeeze()])
  im = norm(im)
  label = transforms.ToTensor()(label).squeeze()
  if torch.sum(label.gt(.003) * label.lt(.004)):
    label *= 255
  label = label.round()

  return im, label


def processTestIm(data):
  (x,y) = data
  im,label = x.copy(), y.copy()
  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])

  # convert to PIL for easier transforms
  im_c1 = Image.fromarray(im[0]).resize((512,512))
  im_c2 = Image.fromarray(im[1]).resize((512,512))
  label = Image.fromarray(label.squeeze()).resize((512,512))

  im_c1s = [F.crop(im_c1, 0, 0, 256, 256), F.crop(im_c1, 0, 256, 256, 256),
            F.crop(im_c1, 256, 0, 256, 256), F.crop(im_c1, 256, 256, 256, 256)]
  im_c2s = [F.crop(im_c2, 0, 0, 256, 256), F.crop(im_c2, 0, 256, 256, 256),
            F.crop(im_c2, 256, 0, 256, 256), F.crop(im_c2, 256, 256, 256, 256)]
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]

  ims = [torch.stack((transforms.ToTensor()(x).squeeze(),
                    transforms.ToTensor()(y).squeeze()))
                    for (x,y) in zip(im_c1s, im_c2s)]

  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)

  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)

  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels = labels.round()

  return ims, labels

Load *flood water* train, test, and validation data from splits. In this example, this is the data we will use to train our model.

In [ ]:
from time import time
import csv
import os
import numpy as np
import rasterio

def getArrFlood(fname):
  return rasterio.open(fname).read()

def download_flood_water_data_from_list(l):
  i = 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    if not os.path.exists(os.path.join("files/", im_fname)):
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("files/", im_fname)))
    arr_y = getArrFlood(os.path.join("files/", mask_fname))
    arr_y[arr_y == -1] = 255

    arr_x = np.clip(arr_x, -50, 1)
    arr_x = (arr_x + 50) / 51

    if i % 100 == 0:
      print(im_fname, mask_fname)
    i += 1
    flood_data.append((arr_x,arr_y))

  return flood_data

def load_flood_train_data(input_root, label_root):
  fname = "flood_train_data.csv"
  training_files = []
  with open(fname) as f:
    for line in csv.reader(f):
      training_files.append(tuple((input_root+line[0], label_root+line[1])))

  return download_flood_water_data_from_list(training_files)

def load_flood_valid_data(input_root, label_root):
  fname = "flood_valid_data.csv"
  validation_files = []
  with open(fname) as f:
    for line in csv.reader(f):
      validation_files.append(tuple((input_root+line[0], label_root+line[1])))

  return download_flood_water_data_from_list(validation_files)

def load_flood_test_data(input_root, label_root):
  fname = "flood_test_data.csv"
  testing_files = []
  with open(fname) as f:
    for line in csv.reader(f):
      testing_files.append(tuple((input_root+line[0], label_root+line[1])))

  return download_flood_water_data_from_list(testing_files)

Load training data and validation data. Note that here, we have chosen to train and validate our model on flood data. However, you can simply replace the load function call with one of the options defined above to load a different dataset.

In [ ]:
train_data = load_flood_train_data('S1/', 'Labels/')
train_dataset = InMemoryDataset(train_data, processAndAugment)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=None,
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
train_iter = iter(train_loader)

valid_data = load_flood_valid_data('S1/', 'Labels/')
valid_dataset = InMemoryDataset(valid_data, processTestIm)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4, shuffle=False, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: (torch.cat([a[0] for a in x], 0), torch.cat([a[1] for a in x], 0)),
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
valid_iter = iter(valid_loader)

S1/Ghana_103272_S1Hand.tif Labels/Ghana_103272_LabelHand.tif
S1/Pakistan_132143_S1Hand.tif Labels/Pakistan_132143_LabelHand.tif
S1/Sri-Lanka_916628_S1Hand.tif Labels/Sri-Lanka_916628_LabelHand.tif
S1/Ghana_5079_S1Hand.tif Labels/Ghana_5079_LabelHand.tif


Define the network. For our purposes, we use ResNet50. However, if you wish to test a different model framework, optimizer, or loss function you can simply replace those here.

In [ ]:
#Esta sección ha sido modificada con la intención de probar distintas arquitecturas,
#así como distintas configuraciones de las mismas.

import torch
import torchvision.models as models
import torch.nn as nn

'''
#ResNet50
net = models.segmentation.fcn_resnet50(pretrained=False, num_classes=2, pretrained_backbone=False)
net.backbone.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=4, bias=False)
'''

'''
#ResNet101 original
net = models.segmentation.fcn_resnet101(pretrained=False, num_classes=2, pretrained_backbone=False)
net.backbone.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
'''


#DeepLab
net = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=2, aux_loss=False)
net.backbone.conv1 = nn.Conv2d(2, 64, kernel_size=8, stride=2, padding=2, bias=False)



criterion = nn.CrossEntropyLoss(weight=torch.tensor([1,8]).float().cuda(), ignore_index=255)
optimizer = torch.optim.AdamW(net.parameters(),lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, len(train_loader) * 10, T_mult=2, eta_min=0, last_epoch=-1)

def convertBNtoGN(module, num_groups=16):
  if isinstance(module, torch.nn.modules.batchnorm.BatchNorm2d):
    return nn.GroupNorm(num_groups, module.num_features,
                        eps=module.eps, affine=module.affine)
    if module.affine:
        mod.weight.data = module.weight.data.clone().detach()
        mod.bias.data = module.bias.data.clone().detach()

  for name, child in module.named_children():
      module.add_module(name, convertBNtoGN(child, num_groups=num_groups))

  return module

net = convertBNtoGN(net)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 73.3MB/s]


Define assessment metrics. For our purposes, we use overall accuracy and mean intersection over union. However, we also include functions for calculating true positives, false positives, true negatives, and false negatives.

In [ ]:
def computeIOU(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()

  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  intersection = torch.sum(output * target)
  union = torch.sum(target) + torch.sum(output) - intersection
  iou = (intersection + .0000001) / (union + .0000001)

  if iou != iou:
    print("failed, replacing with 0")
    iou = torch.tensor(0).float()

  return iou

def computeAccuracy(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()

  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output.eq(target))

  return correct.float() / len(target)

def truePositives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output * target)

  return correct

def trueNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 0)
  correct = torch.sum(output * target)

  return correct

def falsePositives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 1)
  target = (target == 0)
  correct = torch.sum(output * target)

  return correct

def falseNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten()
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 1)
  correct = torch.sum(output * target)

  return correct

Define training loop

In [ ]:
training_losses = []
training_accuracies = []
training_ious = []

def train_loop(inputs, labels, net, optimizer, scheduler):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy

  # zero the parameter gradients
  optimizer.zero_grad()
  net = net.cuda()

  # forward + backward + optimize
  outputs = net(inputs.cuda())
  loss = criterion(outputs["out"], labels.long().cuda())
  loss.backward()
  optimizer.step()
  scheduler.step()

  running_loss += loss
  running_iou += computeIOU(outputs["out"], labels.cuda())
  running_accuracy += computeAccuracy(outputs["out"], labels.cuda())
  running_count += 1

Define validation loop

In [ ]:
valid_losses = []
valid_accuracies = []
valid_ious = []

def validation_loop(validation_data_loader, net):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  global max_valid_iou

  global training_losses
  global training_accuracies
  global training_ious
  global valid_losses
  global valid_accuracies
  global valid_ious

  net = net.eval()
  net = net.cuda()
  count = 0
  iou = 0
  loss = 0
  accuracy = 0
  with torch.no_grad():
      for (images, labels) in validation_data_loader:
          net = net.cuda()
          outputs = net(images.cuda())
          valid_loss = criterion(outputs["out"], labels.long().cuda())
          valid_iou = computeIOU(outputs["out"], labels.cuda())
          valid_accuracy = computeAccuracy(outputs["out"], labels.cuda())
          iou += valid_iou
          loss += valid_loss
          accuracy += valid_accuracy
          count += 1

  iou = iou / count
  accuracy = accuracy / count

  if iou > max_valid_iou:
    max_valid_iou = iou
    save_path = os.path.join("checkpoints", "{}_{}_{}.cp".format(RUNNAME, i, iou.item()))
    torch.save(net.state_dict(), save_path)
    print("model saved at", save_path)

  loss = loss / count
  print("Training Loss:", running_loss / running_count)
  print("Training IOU:", running_iou / running_count)
  print("Training Accuracy:", running_accuracy / running_count)
  print("Validation Loss:", loss)
  print("Validation IOU:", iou)
  print("Validation Accuracy:", accuracy)


  training_losses.append(running_loss / running_count)
  training_accuracies.append(running_accuracy / running_count)
  training_ious.append(running_iou / running_count)
  valid_losses.append(loss)
  valid_accuracies.append(accuracy)
  valid_ious.append(iou)

Define training and validation scheme

In [ ]:
from tqdm.notebook import tqdm
from IPython.display import clear_output

running_loss = 0
running_iou = 0
running_count = 0
running_accuracy = 0

training_losses = []
training_accuracies = []
training_ious = []
valid_losses = []
valid_accuracies = []
valid_ious = []


def train_epoch(net, optimizer, scheduler, train_iter):
  for (inputs, labels) in tqdm(train_iter):
    train_loop(inputs.cuda(), labels.cuda(), net.cuda(), optimizer, scheduler)


def train_validation_loop(net, optimizer, scheduler, train_loader,
                          valid_loader, num_epochs, cur_epoch):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  net = net.train()
  running_loss = 0
  running_iou = 0
  running_count = 0
  running_accuracy = 0

  for i in tqdm(range(num_epochs)):
    train_iter = iter(train_loader)
    train_epoch(net, optimizer, scheduler, train_iter)
  clear_output()

  print("Current Epoch:", cur_epoch)
  validation_loop(iter(valid_loader), net)

Train model and assess metrics over epochs

In [ ]:
#Esta sección ha sido modificada para corregir errores de la versión original
#que impedían la visualización de la gráfica.

import os
from IPython.display import display
import matplotlib.pyplot as plt

max_valid_iou = 0
start = 0

epochs = []
training_losses = []
training_accuracies = []
training_ious = []
valid_losses = []
valid_accuracies = []
valid_ious = []

for i in range(start, 50):
  train_validation_loop(net, optimizer, scheduler, train_loader, valid_loader, 5, i)
  epochs.append(i)
  x = epochs
  plt.plot(x, [l.detach().cpu().numpy() for l in training_losses], label='training losses', color = "#4286f4")
  plt.plot(x, [a.detach().cpu().numpy() for a in training_accuracies], label='training accuracy', color = "#0066cc")
  plt.plot(x, [iou.detach().cpu().numpy() for iou in training_ious], label='training iou', color = "#003d99")
  plt.plot(x, [l.detach().cpu().numpy() for l in valid_losses], label='valid losses', color = "#ff4d4d")
  plt.plot(x, [a.detach().cpu().numpy() for a in valid_accuracies],label='valid accuracy', color = "#cc0000")
  plt.plot(x, [iou.detach().cpu().numpy() for iou in valid_ious],label='valid iou', color = "#990000")
  plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
  display(plt.show())

  print("max valid iou:", max_valid_iou)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

#Secciones añadidas

A partir de aquí el código contenido en este notebook ha sido añadido por Diego Pastor Bonet a modo de complemento del trabajo realizado por Bonafilia et.al.

##Test

Código de autoría propia de Diego Pastor Bonet.
Las funciones de test que se encuentran a continuación han sido creadas basadas en
las de entrenamiento y validación de este mismo cuaderno.

In [ ]:
test_data = load_flood_test_data('S1/', 'Labels/')
test_dataset = InMemoryDataset(test_data, processAndAugment)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=None,
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)

S1/Ghana_313799_S1Hand.tif Labels/Ghana_313799_LabelHand.tif


In [ ]:
test_losses = []
test_accuracies = []
test_ious = []

def test_loop(test_data_loader, net):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  global max_valid_iou

  global training_losses
  global training_accuracies
  global training_ious
  global test_losses
  global test_accuracies
  global test_ious

  net = net.eval()
  net = net.cuda()
  count = 0
  iou = 0
  loss = 0
  accuracy = 0
  with torch.no_grad():
      for (images, labels) in test_data_loader:
          net = net.cuda()
          outputs = net(images.cuda())
          test_loss = criterion(outputs["out"], labels.long().cuda())
          test_iou = computeIOU(outputs["out"], labels.cuda())
          test_accuracy = computeAccuracy(outputs["out"], labels.cuda())
          iou += test_iou
          loss += test_loss
          accuracy += test_accuracy
          count += 1

  iou = iou / count
  accuracy = accuracy / count

  if iou > max_valid_iou:
    max_valid_iou = iou
    save_path = os.path.join("checkpoints", "{}_{}_{}.cp".format(RUNNAME, i, iou.item()))
    torch.save(net.state_dict(), save_path)
    print("model saved at", save_path)

  loss = loss / count
  print("Training Loss:", running_loss / running_count)
  print("Training IOU:", running_iou / running_count)
  print("Training Accuracy:", running_accuracy / running_count)
  print("Test Loss:", loss)
  print("Test IOU:", iou)
  print("Test Accuracy:", accuracy)


  training_losses.append(running_loss / running_count)
  training_accuracies.append(running_accuracy / running_count)
  training_ious.append(running_iou / running_count)
  test_losses.append(loss)
  test_accuracies.append(accuracy)
  test_ious.append(iou)

In [ ]:
def train_test_loop(net, optimizer, scheduler, train_loader,
                          test_loader, num_epochs, cur_epoch):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  net = net.train()
  running_loss = 0
  running_iou = 0
  running_count = 0
  running_accuracy = 0

  for i in tqdm(range(num_epochs)):
    train_iter = iter(train_loader)
    train_epoch(net, optimizer, scheduler, train_iter)
  clear_output()

  print("Current Epoch:", cur_epoch)
  test_loop(iter(test_loader), net)

In [ ]:
import os
from IPython.display import display
import matplotlib.pyplot as plt
max_valid_iou = 0
start = 0

epochs = []
training_losses = []
training_accuracies = []
training_ious = []
test_losses = []
test_accuracies = []
test_ious = []

for i in range(start, 100):
  train_test_loop(net, optimizer, scheduler, train_loader, test_loader, 5, i)
  epochs.append(i)
  x = epochs
  plt.plot(x, [l.detach().cpu().numpy() for l in training_losses], label='training losses', color = "#4286f4")
  plt.plot(x, [a.detach().cpu().numpy() for a in training_accuracies], label='training accuracy', color = "#0066cc")
  plt.plot(x, [iou.detach().cpu().numpy() for iou in training_ious], label='training iou', color = "#003d99")
  plt.plot(x, [l.detach().cpu().numpy() for l in test_losses], label='test losses', color = "#ff4d4d")
  plt.plot(x, [a.detach().cpu().numpy() for a in test_accuracies],label='test accuracy', color = "#cc0000")
  plt.plot(x, [iou.detach().cpu().numpy() for iou in test_ious],label='test iou', color = "#990000")
  plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
  display(plt.show())

  print("max test iou:", max_valid_iou)

Current Epoch: 13


##Gridsearch

Código de autoría propia de Diego Pastor Bonet para el empleo de GridSearch para la optimización de hiperparámetros.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator

class TorchEstimator(BaseEstimator):
    def __init__(self, net, kernel_size, stride):
        self.net = net
        self.kernel_size = kernel_size
        self.stride = stride

    def validation_loop(self, validation_data_loader, net):
        global running_loss
        global running_iou
        global running_count
        global running_accuracy
        global max_valid_iou

        net = net.eval()
        net = net.cuda()
        count = 0
        iou = 0
        loss = 0
        accuracy = 0
        with torch.no_grad():
            for (images, labels) in validation_data_loader:
                net = net.cuda()
                outputs = net(images.cuda())
                valid_loss = criterion(outputs["out"], labels.long().cuda())
                valid_iou = computeIOU(outputs["out"], labels.cuda())
                valid_accuracy = computeAccuracy(outputs["out"], labels.cuda())
                iou += valid_iou
                loss += valid_loss
                accuracy += valid_accuracy
                count += 1

        iou = iou / count
        accuracy = accuracy / count
        '''
        if iou > max_valid_iou:
            max_valid_iou = iou
            save_path = os.path.join("checkpoints", "{}_{}_{}.cp".format(RUNNAME, i, iou.item()))
            torch.save(net.state_dict(), save_path)
            print("model saved at", save_path)
        '''
        loss = loss / count
        print("Training Loss:", running_loss / running_count)
        print("Training IOU:", running_iou / running_count)
        print("Training Accuracy:", running_accuracy / running_count)
        print("Validation Loss:", loss)
        print("Validation IOU:", iou)
        print("Validation Accuracy:", accuracy)

        training_losses.append(running_loss / running_count)
        training_accuracies.append(running_accuracy / running_count)
        training_ious.append(running_iou / running_count)
        valid_losses.append(loss)
        valid_accuracies.append(accuracy)
        valid_ious.append(iou)

    def train_validation_loop(self, optimizer, scheduler, train_loader,
                              valid_loader, num_epochs, cur_epoch):
        global running_loss
        global running_iou
        global running_count
        global running_accuracy
        net = self.net.train()
        running_loss = 0
        running_iou = 0
        running_count = 0
        running_accuracy = 0

        for i in tqdm(range(num_epochs)):
            train_iter = iter(train_loader)
            train_epoch(net, optimizer, scheduler, train_iter)
        clear_output()
        print("Current Epoch:", cur_epoch)
        self.validation_loop(iter(valid_loader), net)

    def fit(self, X, y):
        print("Entrenando la red neuronal...")
        self.net.backbone.conv1.kernel_size = self.kernel_size
        self.net.backbone.conv1.stride = self.stride

        global max_valid_iou
        max_valid_iou = 0
        epochs = []
        training_losses = []
        training_accuracies = []
        training_ious = []
        valid_losses = []
        valid_accuracies = []
        valid_ious = []

        for i in range(0, 5):
            self.train_validation_loop(optimizer, scheduler, train_loader, valid_loader, 1, i)
            epochs.append(i)
            x = epochs
        return self

net = models.segmentation.fcn_resnet50(pretrained=False, num_classes=2, pretrained_backbone=False)
net.backbone.conv1 = nn.Conv2d(2, 64, kernel_size=7, stride=2, padding=3, bias=False)
net = convertBNtoGN(net)
estimator = TorchEstimator(net ,kernel_size=7, stride = 2)

parameters = {'kernel_size': [6, 7]
             # ,'stride' : [1, 2]
              }

X = load_flood_train_data('S1/', 'Labels/')
X = InMemoryDataset(X, processAndAugment)
images = [x[0] for x in X]
labels = [x[1] for x in X]

grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, cv=7, scoring='accuracy')
grid_search.fit(images, labels)

print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Mejor puntuación de validación cruzada:", grid_search.best_score_)

Current Epoch: 4
Training Loss: tensor(0.6970, device='cuda:0', grad_fn=<DivBackward0>)
Training IOU: tensor(0.0104, device='cuda:0')
Training Accuracy: tensor(0.9083, device='cuda:0')
Validation Loss: tensor(0.7261, device='cuda:0')
Validation IOU: tensor(0.0051, device='cuda:0')
Validation Accuracy: tensor(0.8869, device='cuda:0')
Mejores parámetros encontrados: {'kernel_size': 6}
Mejor puntuación de validación cruzada: nan


##SVM

Código de autoría propia de Diego Pastor Bonet para el uso de SVMs como alternativa a las redes neuronales.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

X = load_flood_train_data('S1/', 'Labels/') #Extraemos los datos de las carpetas empleadas anteriormente
X = InMemoryDataset(X, processAndAugment)
images = [x[0] for x in X]
images = [image[0:1] for image in images]
images = [torch.reshape(image, (256, 256)) for image in images]
labels = [x[1] for x in X]

images = images[:20] #Nos quedamos con un número concreto de imágenes debido al elevado tiempo de ejecución
labels = labels[:20]

#-----------------------------------------------------------------------------------------------------------
#Preparación de los datos:

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Aplanar las imágenes para convertirlas en vectores unidimensionales
X_train = np.array([x.numpy().flatten() for x in X_train])
X_test = np.array([x.numpy().flatten() for x in X_test])
y_train = np.array([y.numpy().flatten() for y in y_train])
y_test = np.array([y.numpy().flatten() for y in y_test])

# Aplanar las etiquetas a vectores unidimensionales
X_train = X_train.ravel()
X_test = X_test.ravel()
y_train = y_train.ravel()
y_test = y_test.ravel()

# Convertir los vectores en matrices columna
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

#Comprobamos el balanceo de clases
print('----------------------------------------------------')
print('Datos de entrenamiento:')
unique_values = set(y_train)
count_unique = len(unique_values)
print(f"Valores distintos en y_train: {unique_values}")
print(f"Cantidad de valores distintos: {count_unique}")
count_0 = np.count_nonzero(np.array(y_train) == 0)
count_1 = np.count_nonzero(np.array(y_train) == 1)
print(f"Cantidad de valores 0: {count_0} veces")
print(f"Cantidad de valores 1: {count_1} veces")

print('---------------------------------------------------')
print('Datos de test:')
unique_values = set(y_test)
count_unique = len(unique_values)
print(f"Valores distintos en y_test: {unique_values}")
print(f"Cantidad de valores distintos: {count_unique}")
count_0 = np.count_nonzero(np.array(y_test) == 0)
count_1 = np.count_nonzero(np.array(y_test) == 1)
print(f"Cantidad de valores 0: {count_0} veces")
print(f"Cantidad de valores 1: {count_1} veces")

S1/Ghana_103272_S1Hand.tif Labels/Ghana_103272_LabelHand.tif
S1/Pakistan_132143_S1Hand.tif Labels/Pakistan_132143_LabelHand.tif
S1/Sri-Lanka_916628_S1Hand.tif Labels/Sri-Lanka_916628_LabelHand.tif
----------------------------------------------------
Datos de entrenamiento:
Valores distintos en y_train: {0, 1, 255}
Cantidad de valores distintos: 3
Cantidad de valores 0: 744115 veces
Cantidad de valores 1: 19124 veces
---------------------------------------------------
Datos de test:
Valores distintos en y_test: {0, 1, 255}
Cantidad de valores distintos: 3
Cantidad de valores 0: 179663 veces
Cantidad de valores 1: 4 veces


In [ ]:
#Llevamos a cabo el balanceo de clases:
from sklearn.model_selection import train_test_split

# Contar cuántas muestras de cada clase hay en y_train
count_class_0 = np.sum(y_train == 0)
count_class_1 = np.sum(y_train == 1)

majority_class = 0 if count_class_0 < count_class_1 else 1

indices_clase_0 = np.where(y_train == 0)[0]
indices_clase_1 = np.where(y_train == 1)[0]

# Elegir un número igual de muestras de la clase 0 y la clase 1
num_muestras_clase_0 = min(len(indices_clase_0), len(indices_clase_1))
indices_equilibrados = np.concatenate([indices_clase_0[:num_muestras_clase_0], indices_clase_1[:num_muestras_clase_0]])

# Crear nuevos conjuntos de entrenamiento equilibrados
X_train_equilibrado = X_train[indices_equilibrados]
y_train_equilibrado = y_train[indices_equilibrados]

# Dividir el conjunto equilibrado en entrenamiento y prueba (75% - 25%)
X_train, X_test, y_train, y_test = train_test_split(X_train_equilibrado, y_train_equilibrado, test_size=0.25, random_state=42)

# -------------------------------------------------------------------------------------------
#Verificamos:

# Contar las muestras de cada clase en y_train y y_test
counts_train = np.bincount(y_train)
counts_test = np.bincount(y_test)

# Imprimir el número de muestras de cada clase en y_train
print('---------------------------------------------------')
unique_values = set(y_train)
count_unique = len(unique_values)
print(f"Valores distintos en y_train: {unique_values}")
print(f"Cantidad de valores distintos: {count_unique}")
count_0 = np.count_nonzero(np.array(y_train) == 0)
count_1 = np.count_nonzero(np.array(y_train) == 1)
print(f"Cantidad de valores 0: {count_0} veces")
print(f"Cantidad de valores 1: {count_1} veces")

# Imprimir el número de muestras de cada clase en y_test
print('---------------------------------------------------')
unique_values = set(y_test)
count_unique = len(unique_values)
print(f"Valores distintos en y_test: {unique_values}")
print(f"Cantidad de valores distintos: {count_unique}")
count_0 = np.count_nonzero(np.array(y_test) == 0)
count_1 = np.count_nonzero(np.array(y_test) == 1)
print(f"Cantidad de valores 0: {count_0} veces")
print(f"Cantidad de valores 1: {count_1} veces")

---------------------------------------------------
Valores distintos en y_train: {0, 1}
Cantidad de valores distintos: 2
Cantidad de valores 0: 14335 veces
Cantidad de valores 1: 14351 veces
---------------------------------------------------
Valores distintos en y_test: {0, 1}
Cantidad de valores distintos: 2
Cantidad de valores 0: 4789 veces
Cantidad de valores 1: 4773 veces


In [ ]:
# Iniciar la barra de progreso
with tqdm(total=len(X_train)) as pbar:
    svm_classifier = SVC(kernel='rbf', class_weight={0: 1, 1: 1})
    svm_classifier.fit(X_train, y_train)
    pbar.update(len(X_train))

# Realizar predicciones en los datos de prueba
y_pred = svm_classifier.predict(X_test)

# Calcular la precisión y mostrar un informe de clasificación
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

100%|██████████| 28686/28686 [00:39<00:00, 724.75it/s]


Accuracy: 0.7058146831206861
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.93      0.76      4789
           1       0.88      0.48      0.62      4773

    accuracy                           0.71      9562
   macro avg       0.76      0.71      0.69      9562
weighted avg       0.76      0.71      0.69      9562



###Gridsearch

Es posible utilizar Gridsearch para probar diferentes configuraciones de hiperparámetros de las SVMs. A continuación se proporciona el código para emplear esta herramienta.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Definir los hiperparámetros que deseas probar
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],  # Diferentes kernels
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2}]  # Diferentes configuraciones de pesos
}

# Crear un clasificador SVM
svm_classifier = SVC()

# Crear un objeto GridSearchCV para la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, scoring='accuracy', cv=2)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener la mejor combinación de hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:", best_params)

# Obtener el mejor modelo
best_svm_classifier = grid_search.best_estimator_

# Realizar predicciones en los datos de prueba utilizando el mejor modelo
y_pred = best_svm_classifier.predict(X_test)

# Calcular la precisión y mostrar un informe de clasificación
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Mejores hiperparámetros encontrados: {'class_weight': {0: 1, 1: 1}, 'kernel': 'linear'}
Accuracy: 0.705187199330684
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.91      0.76      4789
           1       0.85      0.50      0.63      4773

    accuracy                           0.71      9562
   macro avg       0.75      0.70      0.69      9562
weighted avg       0.75      0.71      0.69      9562

